In [3]:
import os
import json
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, lower, udf
from pyspark.sql.types import IntegerType, FloatType, ArrayType, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Set environment variable for Hadoop (but we won't use it for saving)
os.environ['HADOOP_HOME'] = 'C:\\Program Files\\hadoop'


# Configure Spark with proper settings for Windows
spark = SparkSession.builder \
    .appName("DarkPatternsDetection") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .config("spark.hadoop.hadoop.native.lib", "false") \
    .getOrCreate()
# Configure Spark with proper settings for Windows

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

In [4]:
# Define keyword dictionaries for dark pattern types
urgency_words = ["limited", "hurry", "soon", "now", "quick", "fast", "expire", "deadline", 
                "flash", "urgent", "today", "last", "ending", "final", "exclusive", "now or never"]

scarcity_words = ["only", "left", "few", "limited", "exclusive", "rare", "stock", "running out", 
                 "almost gone", "remaining", "last chance", "shortage", "sell out", "popular"]

social_proof_words = ["popular", "best seller", "trending", "others", "customers", "reviews", 
                       "rating", "people", "join", "everyone", "trending", "favorite", "recommended","fake"]

# Create functions for pattern detection
def count_matches(text, word_list):
    if not text:
        return 0
    text_lower = text.lower()
    count = 0
    for word in word_list:
        if word.lower() in text_lower:
            count += 1
    return count

# Register UDFs
urgency_udf = udf(lambda text: count_matches(text, urgency_words), IntegerType())
scarcity_udf = udf(lambda text: count_matches(text, scarcity_words), IntegerType())
social_proof_udf = udf(lambda text: count_matches(text, social_proof_words), IntegerType())
text_length_udf = udf(lambda text: len(text.split()) if text else 0, IntegerType())
caps_ratio_udf = udf(lambda t: sum(1 for c in t if c.isupper()) / max(len(t), 1) if t else 0.0, FloatType())
words_udf = udf(lambda text: text.lower().split() if text else [], ArrayType(StringType()))

# Load and preprocess data
df = spark.read.csv("Data/websitedata.tsv", header=True, inferSchema=True, sep="\t")
df = df.drop("page_id")
df = df.na.drop(subset=["text", "label"])
df = df.withColumnRenamed("label", "label_index")
df = df.withColumn("label_index", col("label_index").cast("integer"))
df = df.na.drop(subset=["label_index"])

# Add custom features
df = df.withColumn("text_cleaned", regexp_replace(lower(col("text")), "[^a-zA-Z0-9\\s]", " "))
df = df.withColumn("urgency_score", urgency_udf(col("text")))
df = df.withColumn("scarcity_score", scarcity_udf(col("text")))
df = df.withColumn("social_proof_score", social_proof_udf(col("text")))
df = df.withColumn("text_length", text_length_udf(col("text")))
df = df.withColumn("caps_ratio", caps_ratio_udf(col("text")))

# Instead of using the ML Pipeline's tokenizer which causes conflicts,
# precompute the tokens directly as a feature
df = df.withColumn("words_array", words_udf(col("text_cleaned")))

# Cache the dataframe
df = df.repartition(10)
df.cache()

DataFrame[text: string, label_index: int, Pattern Category: string, text_cleaned: string, urgency_score: int, scarcity_score: int, social_proof_score: int, text_length: int, caps_ratio: float, words_array: array<string>]

In [5]:
# Create TF-IDF features from precomputed words
cv = CountVectorizer(inputCol="words_array", outputCol="word_counts", vocabSize=1000, minDF=2.0)
cv_model = cv.fit(df)
df_counts = cv_model.transform(df)

idf = IDF(inputCol="word_counts", outputCol="tfidf_features")
idf_model = idf.fit(df_counts)
df_tfidf = idf_model.transform(df_counts)

# Train a model using direct approach without saving pipeline
from pyspark.ml.feature import VectorAssembler

# Create feature vector
assembler = VectorAssembler(
    inputCols=["tfidf_features", "urgency_score", "scarcity_score", 
               "social_proof_score", "text_length", "caps_ratio"],
    outputCol="features"
)
df_assembled = assembler.transform(df_tfidf)

# Split data
train, test = df_assembled.randomSplit([0.8, 0.2], seed=42)

# Train LogisticRegression model
lr = LogisticRegression(featuresCol="features", labelCol="label_index", maxIter=20)
lr_model = lr.fit(train)

# Evaluate model
lr_predictions = lr_model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", metricName="accuracy")
accuracy = evaluator.evaluate(lr_predictions)
binary_evaluator = BinaryClassificationEvaluator(labelCol="label_index")
auroc = binary_evaluator.evaluate(lr_predictions)

# Try RandomForest as well
rf = RandomForestClassifier(featuresCol="features", labelCol="label_index", numTrees=20)
rf_model = rf.fit(train)

rf_predictions = rf_model.transform(test)
rf_accuracy = evaluator.evaluate(rf_predictions)
rf_auroc = binary_evaluator.evaluate(rf_predictions)

# Choose the best model
if rf_accuracy > accuracy:
    best_model = rf_model
    best_accuracy = rf_accuracy
    best_auroc = rf_auroc
    model_type = "RandomForest"
else:
    best_model = lr_model
    best_accuracy = accuracy
    best_auroc = auroc
    model_type = "LogisticRegression"

model_dir = "C://Users//manas//OneDrive//Desktop//projects//Big data Project//Current Build//models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_metadata = {
    "model_type": model_type,
    "version": "1.0",
    "accuracy": float(best_accuracy),
    "auc": float(best_auroc),
    "trainDate": "2025-03-01",
    "features": {
        "urgency_words": urgency_words,
        "scarcity_words": scarcity_words,
        "social_proof_words": social_proof_words,
        "countVectorizerVocabSize": 1000
    }
}

if model_type == "LogisticRegression":
    coefficients = best_model.coefficients.toArray().tolist()
    intercept = float(best_model.intercept)
    model_metadata["coefficients"] = coefficients
    model_metadata["intercept"] = intercept
    model_metadata["regParam"] = best_model.getRegParam()
    model_metadata["elasticNetParam"] = best_model.getElasticNetParam()
else:  # RandomForest
    model_metadata["numTrees"] = best_model.getNumTrees()
    model_metadata["maxDepth"] = best_model.getMaxDepth()
    
    # For Random Forest, extract feature importances
    if hasattr(best_model, "featureImportances"):
        importances = best_model.featureImportances.toArray().tolist()
        model_metadata["featureImportances"] = importances

if hasattr(cv_model, "vocabulary"):
    top_vocab = cv_model.vocabulary[:100]  # Save first 100 words
    model_metadata["vocabulary"] = top_vocab

In [6]:
# Save model metadata to JSON
metadata_path = os.path.join(model_dir, "model_metadata.json")
with open(metadata_path, "w") as f:
    json.dump(model_metadata, f)
print(f"Saved model metadata to {metadata_path}")

# Save the model

model_save_path = os.path.join(model_dir, "best_model")
best_model.write().overwrite().save(model_save_path)
print(f"Model saved to {model_save_path}")

Saved model metadata to C://Users//manas//OneDrive//Desktop//projects//Big data Project//Current Build//models\model_metadata.json


Py4JJavaError: An error occurred while calling o648.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:410)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withSco

In [13]:
def predict_dark_patterns(webpage_text):
    input_data = spark.createDataFrame([(webpage_text,)], ["text"])
    input_data = input_data.withColumn("text_cleaned", regexp_replace(lower(col("text")), "[^a-zA-Z0-9\\s]", " "))
    input_data = input_data.withColumn("urgency_score", urgency_udf(col("text")))
    input_data = input_data.withColumn("scarcity_score", scarcity_udf(col("text")))
    input_data = input_data.withColumn("social_proof_score", social_proof_udf(col("text")))
    input_data = input_data.withColumn("text_length", text_length_udf(col("text")))
    input_data = input_data.withColumn("caps_ratio", caps_ratio_udf(col("text")))
    input_data = input_data.withColumn("words_array", words_udf(col("text_cleaned")))

    input_data_counts = cv_model.transform(input_data)
    input_data_tfidf = idf_model.transform(input_data_counts)
    input_data_features = assembler.transform(input_data_tfidf)

    prediction = best_model.transform(input_data_features)
    result = prediction.select("prediction").collect()[0][0]
    return bool(result)

webpage_text = input("Enter the text content of the webpage: ")
contains_dark_patterns = predict_dark_patterns(webpage_text)
print(f"Contains dark patterns: {contains_dark_patterns}")

Enter the text content of the webpage:  bruh


Contains dark patterns: False


In [ ]:
# Function to predict if a webpage contains dark patterns
def predict_dark_patterns(webpage_text):
    input_data = spark.createDataFrame([(webpage_text,)], ["text"])
    input_data = input_data.withColumn("text_cleaned", regexp_replace(lower(col("text")), "[^a-zA-Z0-9\\s]", " "))
    input_data = input_data.withColumn("urgency_score", urgency_udf(col("text")))
    input_data = input_data.withColumn("scarcity_score", scarcity_udf(col("text")))
    input_data = input_data.withColumn("social_proof_score", social_proof_udf(col("text")))
    input_data = input_data.withColumn("text_length", text_length_udf(col("text")))
    input_data = input_data.withColumn("caps_ratio", caps_ratio_udf(col("text")))
    input_data = input_data.withColumn("words_array", words_udf(col("text_cleaned")))

    input_data_counts = cv_model.transform(input_data)
    input_data_tfidf = idf_model.transform(input_data_counts)
    input_data_features = assembler.transform(input_data_tfidf)

    prediction = best_model.transform(input_data_features)
    result = prediction.select("prediction").collect()[0][0]
    return bool(result)

# Example usage
webpage_text = input("Enter the text content of the webpage: ")
contains_dark_patterns = predict_dark_patterns(webpage_text)
print(f"Contains dark patterns: {contains_dark_patterns}")

In [ ]:
What is Lorem Ipsum? Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.  Why do we use it? It is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution of letters, as opposed to using 'Content here, content here', making it look like readable English. Many desktop publishing packages and web page editors now use Lorem Ipsum as their default model text, and a search for 'lorem ipsum' will uncover many web sites still in their infancy. Various versions have evolved over the years, sometimes by accident, sometimes on purpose (injected humour and the like).   Where does it come from? Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit amet..", comes from a line in section 1.10.32.  The standard chunk of Lorem Ipsum used since the 1500s is reproduced below for those interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et Malorum" by Cicero are also reproduced in their exact original form, accompanied by English versions from the 1914 translation by H. Rackham.  Where can I get some? There are many variations of passages of Lorem Ipsum available, but the majority have suffered alteration in some form, by injected humour, or randomised words which don't look even slightly believable. If you are going to use a passage of Lorem Ipsum, you need to be sure there isn't anything embarrassing hidden in the middle of text. All the Lorem Ipsum generators on the Internet tend to repeat predefined chunks as necessary, making this the first true generator on the Internet. It uses a dictionary of over 200 Latin words, combined with a handful of model sentence structures, to generate Lorem Ipsum which looks reasonable. The generated Lorem Ipsum is therefore always free from repetition, injected humour, or non-characteristic words etc.

In [ ]:
Skip to Main content About this item About this item Buying options Compare with similar items Reviews Keyboard shortcuts Search alt + / Cart shift + alt + C Home shift + alt + H Orders shift + alt + O  Add to cart shift + alt + K  Show/Hide shortcuts shift + alt + Z To move between items, use your keyboard's up or down arrows. .in Delivering to Vellore 632106 Update location Home & Kitchen  Home & Kitchen Search Amazon.in EN Hello, sign in Account & Lists Returns & Orders 0 Cart All Fresh MX Player Sell Bestsellers Today's Deals Mobiles Prime Customer Service New Releases Electronics Fashion Amazon Pay Home & Kitchen Computers Books Car & Motorbike Toys & Games Home Improvement Beauty & Personal Care Sports, Fitness & Outdoors Gift Cards Custom Products Grocery & Gourmet Foods Health, Household & Personal Care Baby Video Games Pet Supplies Gift Ideas Audible AmazonBasics Subscribe & Save Kindle eBooks Amazon Home Kitchen & Home Appliances Large Appliances Kitchen & Dining Furniture Home Furnishing Home Decor Home Improvement Garden & Outdoor Storage & Organisation Lighting Home & Kitchen›Heating, Cooling & Air Quality›Air Conditioners›Split-System Air Conditioners ₹36,990.00 Fulfilled FREE scheduled delivery as soon as Friday, 14 March, 7 am - 9 pm. Order within 18 hrs 48 mins. Details Delivering to Vellore 632106 - Update location In stock Payment Secure transaction Ships from Amazon Sold by DAWNTECH ELECTRONICS PRIVATE LIMITED Payment Secure transaction We work hard to protect your security and privacy. Our payment security system encrypts your information during transmission. We don’t share your credit card details with third-party sellers, and we don’t sell your information to others. Learn more Ships from Amazon Sold by DAWNTECH ELECTRONICS PRIVATE LIMITED Include Add a Protection Plan: Additional AC warranty for 1 Year for ₹899.00 Damage Protection & Warranty combo for ₹1,788.00 Quantity: 1 Quantity:1 Add to Cart Buy Now Add gift options Add to Wish List Other sellers on Amazon New (2) from ₹36,990.00   Fulfilled FREE Scheduled Delivery   5 VIDEOS     2+ Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, Triple Display, Dew Clean Technology, Coanda Airflow, 2024 Model, MTKL50U, White) Roll over image to zoom in Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, Triple Display, Dew Clean Technology, Coanda Airflow, 2024 Model, MTKL50U, White) Visit the Daikin Store 3.9 3.9 out of 5 stars    4,023 ratings | Search this page #1 Best Seller in Split-System Air Conditioners 4K+ bought in past month ₹36,990.00 with 37 percent savings-37% ₹36,990 M.R.P.: ₹58,400.00M.R.P.: ₹58,400 Fulfilled Inclusive of all taxes EMI starts at ₹1,793. No Cost EMI available EMI options    Offers No Cost EMI Upto ₹2,903.15 EMI interest savings on select Credit Cards, HDFC Bank Debit Cards, Amazon Pay LaterUpto ₹2,903.15 EMI interest savings on select Credit Cards… 3 offers Bank Offer Upto ₹1,500.00 discount on select Credit Cards, Debit CardsUpto ₹1,500.00 discount on select Credit Cards, Debit… 36 offers Partner Offers Get GST invoice and save up to 28% on business purchases. Sign up for freeGet GST invoice and save up to 28% on business purchases. 1 offer Paid Installation available  Paid Installation available 1 Year Warranty Care 1 Year Warranty Care 10 days Replacement by Brand 10 days Replacement by Brand Free Delivery Free Delivery Amazon Delivered Amazon Delivered Top Brand Top Brand Next page Style Name: 2024 - 1.5 Ton 3 Star 2024 - 1.5 Ton 3 Star ₹36,990.00 ₹58,400.00 1.5 Ton 3 star-2023 Model ₹39,990.00 ₹64,990.00 2022 - 1.5 Ton 3 Star Inv Currently unavailable.  Pay INR 699 + GST for standard installation directly to the brand technician; book now pay during serviceDetails Price:₹0.003.3 stars (1338)See more Brand	Daikin Capacity	1.5 Tons Cooling Power	17100 British Thermal Units Special Feature	High Ambient Operation upto 52°C, 3D Airflow, Dew Clean Technology, Triple Display, PM 2.5 Filter Product Dimensions	22.9D x 88.5W x 29.8H Centimeters About this item split ac with inverter swing compressor and hepta sense: dew clean technology; triple display; pm 2.5 filter to ensure healthy air capacity 1.5 ton: suitable for small sized rooms (111 to 150 sq.ft); 572 cfm with an air throw of 16 meters 3 star: energy efficiency warranty: 1 years on product, 5 years on pcb, 10 years on compressor copper condenser coil with patented dnns self heal coating for low maintenance & enhanced durability key features- cooling capacity @ 43°c: 100%; noise level: 30 db(a); ambient operation: 52°c special features: auto variable speed, triple display (power consumption %age, set/room temperature & auto error code); 3d airflow for uniform cooling and power chill for faster cooling Show More    Report an issue with this product  Explore home, kitchen & more products from Solimo. Premium Quality. Great Value. Solimo offers a range of kitchenware, home furnishing, mobile accessories, appliances, home decor, backpacks, mattresses and much more. Shop now Top Brand  Daikin   83% positive ratings from 10K+ customers   10K+ recent orders from this brand   10+ years on Amazon  Frequently bought together  Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, Triple Display, Dew Clean Technology, Coanda Airflow, 2024 Mo This item: Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, Triple Display, Dew Clean Technology, Coanda Airflow, 2024 Model, MTKL50U, White) ₹36,990.00 +  Mumma's LIFE AC Stand/Heavy Duty Air Conditioner Outdoor Unit Mounting Bracket (White) Mumma's LIFE AC Stand/Heavy Duty Air Conditioner Outdoor Unit Mounting Bracket (White) ₹498.00 Total price: ₹37,488.00 Add both to Cart One of these items is dispatched sooner than the other. Show details Related products with free delivery on eligible orders Sponsored  Daikin 1.5 Ton 5 Star Inverter Split AC (Copper, PM 2.5 Filter, MTKM50U, White) Daikin 1.5 Ton 5 Star Inverter Split AC (Copper, PM 2.5 Filter, MTKM50U, White)  2,700 -32%₹45,490.00 M.R.P:₹67,200.00   Daikin 1 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, 2024 Model, MTKL35UV16, White) Daikin 1 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, 2024 Model, MTKL35UV1...  1,064 -31%₹33,490.00 M.R.P:₹48,200.00   LG 1.5 Ton 3 Star DUAL Inverter Split AC (Copper, AI Convertible 6-in-1, VIRAAT Mode, Diet Mode+, Faster Cooling & Energy Saving, HD Filter with Anti-Virus Protection, US-Q18JNXE, White) LG 1.5 Ton 3 Star DUAL Inverter Split AC (Copper, AI Convertible 6-in-1, VIRAAT Mod...  979 -52%₹37,690.00 M.R.P:₹78,990.00   Daikin 0.8 Ton 3 Star, Fixed Speed Split AC (Copper, PM 2.5 Filter, 2022 Model, FTL28U, White) Daikin 0.8 Ton 3 Star, Fixed Speed Split AC (Copper, PM 2.5 Filter, 2022 Model, FTL...  1,660 -29%₹26,490.00 M.R.P:₹37,400.00   Carrier 1.5 Ton 3 Star Wi-Fi Smart Flexicool Inverter Split AC (Copper, Convertible 6-in-1 Cooling,Smart Energy Display, HD & PM 2.5 Filter, ESTER EDGE FXi (Wi-Fi), CAI18EE3R35W0,White) Carrier 1.5 Ton 3 Star Wi-Fi Smart Flexicool Inverter Split AC (Copper, Convertible...  1,657 -48%₹35,490.00 M.R.P:₹68,790.00   Panasonic 1.5 Ton 3 Star Inverter Split AC (Copper Condenser, 7 in 1 Convertible, PM 0.1 Air Purification Filter, Hidden Display, CS/CU-SU18ZKYTK, White) Panasonic 1.5 Ton 3 Star Inverter Split AC (Copper Condenser, 7 in 1 Convertible, P...  153 -38%₹33,990.00 M.R.P:₹54,400.00   Panasonic 1.5 Ton 5 Star Premium Wi-Fi Inverter Smart Split AC (Matter Enabled, Higher Airflow, Copper Condenser, 7in1 Convertible, True AI, 4-Way, PM 0.1 Filter, CS/CU-NU18AKY5WX, White) Panasonic 1.5 Ton 5 Star Premium Wi-Fi Inverter Smart Split AC (Matter Enabled, Hig...  3,934 Amazon's  Choice in Split-System Air Conditioners -30%₹44,990.00 M.R.P:₹64,400.00 Product information Technical Details Brand	‎Daikin Model	‎MTKL50U Energy Efficiency	‎3 star Capacity	‎1.5 Tons Annual Energy Consumption	‎966.47 Kilowatt Hours Per Year Noise Level	‎35 dB Installation Type	‎Split System Part Number	‎MTKL50U Form Factor	‎Mini-Split Special Features	‎High Ambient Operation upto 52°C, 3D Airflow, Dew Clean Technology, Triple Display, PM 2.5 Filter Colour	‎White Control Console	‎Remote Control Voltage	‎230 Volts Wattage	‎966.47 Kilowatt Hours Certification	‎Energy Star Material	‎IDU: HIPS (High grade plastic) ; ODU: HDZP Steel Sheet Included Components	‎1 Indoor Unit, 1 Outdoor unit, 1 Remote Controller, 3 Mtr. Length Inter Connecting Copper Coil Pipe, 1 Mounting plate, 2 Accessory Filter, 2.95 Mtr. Insulation Pipe, 3.7 Mtr. Connecting cable (wire), 2 Tie Rap & 9 fixing screws. Scan the QR code on manufacturing label for soft copy of Operational Manual, Installation Manual, Warranty Card, E-Waste Guideline & Wiring Diagram. Batteries Included	‎No Batteries Required	‎Yes Battery Cell Type	‎Zinc Manufacturer	‎Daikin airconditioning India Pvt. Ltd., 210, 1st Floor, Okhla Industrial Area, Phase 3, Delhi 110020 Country of Origin	‎India Additional Information ASIN	B0BK1KS6ZD Customer Reviews	3.9 3.9 out of 5 stars    4,023 ratings 3.9 out of 5 stars Best Sellers Rank	#98 in Home & Kitchen (See Top 100 in Home & Kitchen) #1 in Split-System Air Conditioners Date First Available	21 October 2022 Manufacturer	Daikin airconditioning India Pvt. Ltd., 210, 1st Floor, Okhla Industrial Area, Phase 3, Delhi 110020, Daikin Airconditioning India Pvt. Ltd., Daikin airconditioning India Pvt. Ltd., 210, 1st Floor, Okhla Industrial Area, Phase 3, Delhi 110020 Packer	Daikin airconditioning India Pvt. Ltd., 210, 1st Floor, Okhla Industrial Area, Phase 3, Delhi 110020 Importer	Daikin airconditioning India Pvt. Ltd., 210, 1st Floor, Okhla Industrial Area, Phase 3, Delhi 110020 Item Weight	31 kg Item Dimensions LxWxH	22.9 x 88.5 x 29.8 Centimeters Net Quantity	1 Count Generic Name	Split airconditioner Feedback Would you like to tell us about a lower price?  What is in the box? 1 Indoor Unit, 1 Outdoor unit, 1 Remote Controller, 3 Mtr. Length Inter Connecting Copper Coil Pipe, 1 Mounting plate, 2 Accessory Filter, 2.95 Mtr. Insulation Pipe, 3.7 Mtr. Connecting cable (wire), 2 Tie Rap & 9 fixing screws. Scan the QR code on manufacturing label for soft copy of Operational Manual, Installation Manual, Warranty Card, E-Waste Guideline & Wiring Diagram. From the manufacturer MTKL50UV16, MTKL35UV16 Dew Clean Technology  Dew Clean Technology is a one button operation, that, when activated, automatically cleans the evaporator coil of the indoor unit heat exchanger by utilizing condensate water. This allows for better cleaning of Indoor Unit Heat Exchanger which ensures better airflow & consistent cooling performance throughout the year.  PM 2.5 Filter  It is capable to Trap fine air particles upto 2.5 microns resulting clean and pure inside air.  Triple Display  Are you sure inverter ACs, save money? Daikin's inverter technology is best in class because it has the widest range of compressor frequencies. You can witness the same with Daikin’s triple display feature. It enables you to witness power saving due to inverter technology in form of estimated power consumption percentage of your AC in at any given point in time, on real time basis along with set/room temperature and error code.  3D Airflow  3D Airflow ensures 4 way automatic uniform cooling at every corner of your room for a great air conditioning experience.  Stabilizer Inside  This machine is equipped with Stabilizer inside which removes its dependency from the external stabilizer. This is different from stabilizer free operation as it regulates voltage too  Econo Mode  Econo mode enables efficient operation by limiting the maximum power consumption. It is useful when using the air-conditioner and other electrical devices simultaneously on a shared electrical circuit. This mode helps you in reducing your electricity bills.  Power Chill  Power chill operation of your AC ensures instant and faster cooling even in the extreme heat conditions. It cools 20% faster than the normal mode providing instant relief from the heat.  R-32  R32 (Green Refrigerant) has zero Ozone Depletion Potential (ODP) and Modified Global Warming Potential (GWP) of 472, compared to R410A’s Modified GWP of 2,027. Also R32 is a single component refrigerant, which makes it easy to recycle. It is because of these reasons that R32 offers the lowest total emissions and best overall life cycle climate performance.  100% Copper Coil  Daikin air conditioners Condensers are equipped with 100% copper coils which functions as the backbone for robust air conditioning system.  Self Diagnosis  It auto detects the error and shows the related code on the remote screen. Now you can easily self diagnose the error and report it to your Daikin authorized service center for a quick resolution.  Coanda Airflow  Radiant cooling enhances comfort cooling by evenly distributing air in every corner of the room. With the help of one click on your remote control, experience enhanced comfort.  Good Sleep Off Timer  Good sleep off timer option empowers you to control temperature of your air conditioner while you are sleeping. You can adjust room temperature as per your comfort.  Compare with similar items This Item Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, Triple Display, Dew Clean Technology, Coanda Airflow, 2024 Model, MTKL50U, White) Daikin 1.5 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, Triple Display, Dew Clean Technology, Coanda Airflow, 2024 Model, MTKL50U, White) Recommendations Lloyd 1.5 Ton 3 Star Inverter Split AC (5 in 1 Convertible, Copper, Anti-Viral + PM 2.5 Filter,White with Chrome Deco Strip, GLS18I3FWAGC) Lloyd 1.5 Ton 3 Star Inverter Split AC (5 in 1 Convertible, Copper, Anti-Viral + PM 2.5 Filter,White with Chrome Deco Strip, GLS18I3FWAGC)   Hitachi 1.5 Ton Class 3 Star, ice Clean, Xpandable+, Inverter Split AC (100% Copper, Dust Filter, 3400FXL RAS.G318PCBIBF, White) Hitachi 1.5 Ton Class 3 Star, ice Clean, Xpandable+, Inverter Split AC (100% Copper, Dust Filter, 3400FXL RAS.G318PCBIBF, White)   Daikin 1.5 Ton 3 Star Inverter Split AC (F/A/DTKL50U) White Daikin 1.5 Ton 3 Star Inverter Split AC (F/A/DTKL50U) White Add to cart Add to cart Add to cart Add to cart Price	-37% ₹36,990.00 M.R.P: ₹58,400.00	-42% ₹34,490.00 M.R.P: ₹58,990.00	-41% ₹36,990.00 M.R.P: ₹63,100.00	-36% ₹37,490.00 M.R.P: ₹58,400.00 Delivery	 Get it by Friday, March 14 Get it by Friday, March 14 Get it by Friday, March 14 Get it by Friday, March 14 Customer Ratings	 3.93.9 out of 5 stars4,023 4.14.1 out of 5 stars618 4.24.2 out of 5 stars526 3.83.8 out of 5 stars6 Sold by	 DAWNTECH ELECTRONICS PRIVATE LIMITED DAWNTECH ELECTRONICS PRIVATE LIMITED DAWNTECH ELECTRONICS PRIVATE LIMITED Official Seller - India cooling power	 17100 btus 4.75 kilowatts 5 kilowatts 5500 btus floor area	 2637 square centimeters 160 square feet — — control method	 remote remote remote — power source	 Corded Electric Corded Electric Corded Electric Corded Electric special feature	 PM 2.5 Filter, Triple Display, Dew Clean Technology, High Ambient Operation upto 52°C, 3D Airflow 5 in 1 Convertible; Cools Even @ 52 ˚C, Turbo Cool; 100% Copper, Golden Fin Evaporator; Anti-Viral Filter + PM 2.5 Filter, Low Gas Detection; Clean Filter Indication, Installation Check; Stabilizer Free Operation; Hidden LED Display ice Clean (FrostWash), Xpandable+, Ambience Light, Hexa Sensor, Odour-free air Dust Filter energy star	 3 Star 3 Star 3 Star 3 Star model name	 MTKL50U GLS18I3FWAGC AC — voltage	 230 volts 230 volts 230 volts 230 volts of alternating current wattage	 966.47 kilowatt hours 1565 watts 1940 watts 240 kilowatt hours air flow efficiency	 — 382 cubic feet per minute per watt 530 cubic feet per minute per watt — SEER rating	 4 3.84 3.82 — Products related to this item Sponsored  Daikin 1 Ton 5 Star Inverter Split AC (Copper, PM 2.5 Filter, 2022 Model, MTKM35U, White) Daikin 1 Ton 5 Star Inverter Split AC (Copper, PM 2.5 Filter, 2022 Model, MTKM35U, ...  759 -30%₹38,990.00 M.R.P:₹55,500.00   Carrier 1.5 Ton 5 Star Wi-Fi Smart Flexicool Inverter Split AC (Copper, Convertible 6-in-1 Cooling,Smart Energy Display,HD & PM 2.5 Filter, ESTER EDGE FXi (Wi-Fi), CAI19EE5R35W0,White) Carrier 1.5 Ton 5 Star Wi-Fi Smart Flexicool Inverter Split AC (Copper, Convertible...  1,717 -44%₹42,990.00 M.R.P:₹76,090.00   Panasonic 1.5 Ton 3 Star Premium Wi-Fi Inverter Smart Split AC (Matter Enabled, Higher Airflow, Copper Condenser, 7in1 Convertible, True AI, 2-Way, PM 0.1 Filter, CS/CU-SU18AKY3W, White) Panasonic 1.5 Ton 3 Star Premium Wi-Fi Inverter Smart Split AC (Matter Enabled, Hig...  3,958 Amazon's  Choice in Split-System Air Conditioners -34%₹37,490.00 M.R.P:₹56,400.00   Midea 2 Ton 3 Star Smart Wi-Fi AI Gear Inverter Split AC (Copper, Convertible 4-in-1 Cooling,HD Filter with Auto Cleanser, SANTIS NEO DLX (Wi-Fi), MAI24SD3R34W0,White) Midea 2 Ton 3 Star Smart Wi-Fi AI Gear Inverter Split AC (Copper, Convertible 4-in-...  148 -40%₹42,990.00 M.R.P:₹71,790.00   LG 1.5 Ton 3 Star DUAL Inverter Split AC (Copper, AI Convertible 6-in-1, VIRAAT Mode, Diet Mode+, Faster Cooling & Energy Saving, HD Filter with Anti-Virus Protection, US-Q18JNXE, White) LG 1.5 Ton 3 Star DUAL Inverter Split AC (Copper, AI Convertible 6-in-1, VIRAAT Mod...  979 -52%₹37,690.00 M.R.P:₹78,990.00   Daikin 1 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, 2024 Model, MTKL35UV16, White) Daikin 1 Ton 3 Star Inverter Split AC (Copper, PM 2.5 Filter, 2024 Model, MTKL35UV1...  1,064 -31%₹33,490.00 M.R.P:₹48,200.00   Daikin 0.8 Ton 3 Star, Fixed Speed Split AC (Copper, PM 2.5 Filter, 2022 Model, FTL28U, White) Daikin 0.8 Ton 3 Star, Fixed Speed Split AC (Copper, PM 2.5 Filter, 2022 Model, FTL...  1,660 -29%₹26,490.00 M.R.P:₹37,400.00 Looking for specific info? Search in reviews, Q&A... Customer reviews 3.9 out of 5 stars 3.9 out of 5 4,023 global ratings 5 star 55% 4 star 21% 3 star 6% 2 star 2% 1 star 16% How are ratings calculated? Review this product Share your thoughts with other customers Write a product review Customers say Customers find the air conditioner to be a good product with a good cooling feature. They appreciate its value for money and energy efficiency. However, some customers dislike the service quality. Opinions vary on installation, performance, and noise level.  AI-generated from the text of customer reviews  Select to learn more Quality Cooling Value for money Energy efficiency Installation Performance Noise level Service quality Reviews with images See all photos  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review  Customer Image, click to open customer review Top reviews from India  Shambhu Chowdhary 5.0 out of 5 stars It's worth every penny! Reviewed in India on 8 March 2025 Style Name: 2024 - 1.5 Ton 3 StarVerified Purchase I recently purchased the Daikin 1.5 Ton 3 Star Inverter Split AC, and I couldn't be happier with my decision. This AC is a game-changer for several reasons:  Efficient Cooling: The cooling performance is outstanding. It cools my room quickly and maintains a consistent temperature, even during the hottest days. Energy Saving: The inverter technology ensures that the AC runs efficiently, saving on electricity bills without compromising on performance. Quiet Operation: The AC operates quietly, making it perfect for a peaceful night's sleep. Durable Build: The copper condenser coil enhances durability and ensures long-lasting performance. User-Friendly Features: The remote control is easy to use, and features like the sleep mode and timer add to the convenience. Overall, I highly recommend the Daikin 1.5 Ton 3 Star Inverter Split AC for anyone looking for a reliable and efficient cooling solution. It's worth every penny! Helpful Report  Aditya Gupta🔥 4.0 out of 5 stars High quality product from Daikin Reviewed in India on 3 April 2024 Style Name: 2024 - 1.5 Ton 3 StarVerified Purchase The delivery was quick and the AC was delivered 3 days before the date which is impressive. I got sms about installation booking on the same day and AC was installed next day. Installation was conducted by two teenage boys who were quiet inexperienced and took almost 5 hours in the whole process, they did try to sell their ODU stand @ ₹850 but I had already arranged iron angle stand made by local iron works shop for about half the price and they (installation guys) installed the ODU on my own stand.  They drilled a hole in the wall for the copper pipes and did not fill it with anything to shut it so I'll have to close it myself with wall putty or something else.  The installation team did not overcharge me for anything as I arranged everything on my own for less than half of their price.  Overall the installation was OK and very cheap (825 only)  Now come to AC after installation.  The AC takes quite time to start working but after that time the AC cools very fast and manages the temperature of my large room (230sqft)  Pros- 1- Four way swing  2- Air throw is very powerful  3- Indoor unit makes very less noise  4- The matte finish looks great  Cons-  1- Non convertible AC  2- Outdoor unit is very noisy  3- No batteries with remote  4- No user manual  As of now I've used it for just 30 minutes, will update after few days where the temperature rises. Read more Customer image 95 people found this helpful Helpful Report  Nitish Singh 5.0 out of 5 stars One of the best AC out there + excellent remote? Reviewed in India on 6 April 2019 Verified Purchase If you are looking for a complete package, then look nowhere other than Daikin.  I purchased the 1.5 Ton 4 Star AC 10-days ago, and until now I am impressed with what it has to offer. To give you guys a better idea, let me go through the whole experience.  Installation: Installation went super smooth. The installation guys called me even before the AC arrived. I told them that it had not arrived yet. They insisted on letting them know once it does. So, once the AC arrived, I called them, and to my surprise, they said that they are coming right away.  The installation process started, and it was done in 3 hours max.  Also, for those people who are saying that they are charging money even when installation is free, you guys have no idea about how AC installation is done.  According to the offer when I purchased the AC, the installation charges are free. This means that the engineers won't take labor charges for installing the AC. But, that doesn't mean that you also don't have to pay for the extra materials that are required to install the AC!  Also, the extra charges can vary heavily depending on the amount of material needed including pipe, tape and so on. In my case, it cost me 2500 INR, and I am super happy with it.  Build Quality and Design The build quality of the AC is excellent. The internal unit looks sleek and modern. However, I do have a complaint here. The bottom part of the internal unit slightly bugles during installation. According to engineer, it cannot be fixed (or removed) due to the backplate that is fixed on the wall. The AC resets on the backplate. Anyway, it is not a big issue, and hard to notice for visitors..so I agreed to not do anything else.  Functionality and Remote The AC functions as advertised. It cools and cools very nicely. However, I was not expecting that the AC remote would be so awesome! It has almost every functionality that you would need. Few of my key features that can be controlled through remote include Coanda mode which works in conjunction with a fan. You just turn it on and put a fan on the slowest speed. Your room will be filled with a cold ambiance unlike direct cold air in case of swing fans. I also love the display at the remote. It shows all the possible icons and notification that makes handling AC a charm! I also loved Good Slepe mode, Econo mode, and horizontal and vertical swing!  Manual The manual is also nicely crafted. I used it to learn about the AC which the engineers fail to tell me. I told the installation guys to give me a demo, but they couldn't. I don't blame them as they just basic stuff. I would request everyone who buys the AC to read the manual at least once to understand the true functionality of the AC.  Overall Value The overall value of the AC is just excellent. It is affordable and enables you to enjoy with minimal electricity bills(I am skeptical here as I am yet to receive the first electric bill, but I think it will be fine)!  Summary 1. Excellent build quality. 2. Out of the world AC remote. 3. Great feature-set 4. No display on the AC unit(CON) 5. Excellent manual  Overall rating: 9.5/10  About the service, I am not sure as I am yet to take advantage of it. However, from my friend circle and peers, they are well known for their service(at least in the Kolkata region.) Read more Customer imageCustomer imageCustomer image 140 people found this helpful Helpful Report See more reviews ­ ­ Back to top Get to Know Us About Amazon Careers Press Releases Amazon Science Connect with Us Facebook Twitter Instagram Make Money with Us Sell on Amazon Sell under Amazon Accelerator Protect and Build Your Brand Amazon Global Selling Supply to Amazon Become an Affiliate Fulfilment by Amazon Advertise Your Products Amazon Pay on Merchants Let Us Help You Your Account Returns Centre Recalls and Product Safety Alerts 100% Purchase Protection Amazon App Download Help   English India AbeBooks Books, art & collectibles Amazon Web Services Scalable Cloud Computing Services Audible Download Audio Books IMDb Movies, TV & Celebrities Shopbop Designer Fashion Brands Amazon Business Everything For Your Business Prime Now 2-Hour Delivery on Everyday Items Amazon Prime Music 100 million songs, ad-free Over 15 million podcast episodes Conditions of Use & Sale Privacy Notice Interest-Based Ads © 1996-2025, Amazon.com, Inc. or its affiliates


# WebScraping

In [ ]:
pip install requests beautifulsoup4

In [15]:
import requests
from bs4 import BeautifulSoup

def scrape_page(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {url}")
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    webpage_text = soup.get_text(separator=' ', strip=True)
    # print(webpage_text)
    return webpage_text
def scrape_and_predict(urls):
    results = {}
    for url in urls:
        webpage_text = scrape_page(url)
        if webpage_text:
            contains_dark_patterns = predict_dark_patterns(webpage_text)
            results[url] = contains_dark_patterns
    return results

urls = [
    "https://www.lipsum.com/",
    # "https://www.amazon.in/Daikin-Inverter-Display-Technology-MTKL50U/dp/B0BK1KS6ZD/ref=sr_1_2?_encoding=UTF8&content-id=amzn1.sym.58c90a12-100b-4a2f-8e15-7c06f1abe2be&dib=eyJ2IjoiMSJ9.LpujZ4uISPUK8sa_6yNGVTLp2_seTR9samDUOPD7O24PE2kANyrymdZzhCoNMjav4RVZpWfhS7NpfmT_1QrlbWK8IvTfXwV-qMalaE9wpOkQiW1Wl2-7r1lW2el2hgSS4-Vp0CvRYV2-HedAeu4HGDoDBdMN-FgO_NTgY_Y4z-qsOH4_cGCi4MaI5au3m1xprFGO4Y3z4TQxRrCaGjkW6QoVWai9ptEtd96K_Sac5QQ2HbvrckkgZRQUZz8F78OAd-o7_f6m5u0_hDv8-Q6lNh8wVeXzpEwE9_QCM_vSKDI.J2JkzduIQN1BWCiZlePcSwyqnUY_FnkkOfMyYedB5mk&dib_tag=se&pd_rd_r=2561953e-6359-4825-8748-1d5d2d584deb&pd_rd_w=e8wqo&pd_rd_wg=8gWP8&qid=1743102411&refinements=p_85%3A10440599031&rps=1&s=kitchen&sr=1-2&th=1"
    "https://www.reddit.com/r/darkpatterns/comments/10tubkr/what_are_some_examples_of_dark_patterns_on/"
    # "https://www.amazon.in/Daikin-Inverter-Display-Technology-MTKL50U/dp/B0BK1KS6ZD/ref=sr_1_3?_encoding=UTF8&content-id=amzn1.sym.58c90a12-100b-4a2f-8e15-7c06f1abe2be&dib=eyJ2IjoiMSJ9.LpujZ4uISPUK8sa_6yNGVTLp2_seTR9samDUOPD7O248t3UcPIyFUOP6Gv_tzBZVrrNsPUqaO56EB1q2ZtYqEXR0PVk7H6WSd61ZKLSMOvDX9pXRa2zFEf5cDwl__f6l5OukJv-waLYXhMtvVncsZmdeZgwAnFI-F_0TdMzSMFirmhVGOA-OSYvLbBZTrf2BQ0hy868foC-MdsmyM1b8YkZGVXf8dRELdkAJ-BwLOl_4e35o-RC3GBBBziD50-IzEh2klz9JF_vqpiFpAS4WZ9cViCB_8k1l7vhC2f-MJ2k.0YKmRsGGOQeSxv-KFYZ2xdfDBoVfU0uhMP4dPK2eMb4&dib_tag=se&pd_rd_r=91c66077-0483-4ba3-b186-5aa3850abef5&pd_rd_w=6tsQP&pd_rd_wg=DTJ50&qid=1742758995&refinements=p_85%3A10440599031&rps=1&s=kitchen&sr=1-3&th=1",
]

results = scrape_and_predict(urls)
for url, contains_dark_patterns in results.items():
    print(f"URL: {url} - Contains dark patterns: {contains_dark_patterns}")

URL: https://www.lipsum.com/ - Contains dark patterns: False
URL: https://www.reddit.com/r/darkpatterns/comments/10tubkr/what_are_some_examples_of_dark_patterns_on/ - Contains dark patterns: False


In [27]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
def setup_driver():
    """Set up and return a configured Chrome webdriver."""
    options = Options()
    options.add_argument("--headless")  # Run without opening browser
    options.add_argument("--disable-blink-features=AutomationControlled")  # Bypass bot detection
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # Set user agent to appear more like a regular browser
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver
def scrape_page(url, driver):
    """Scrape a webpage and return its text content using Selenium."""
    try:
        driver.get(url)
        # Wait for the page to load
        time.sleep(5)
        
        # Get the page content
        webpage_text = driver.find_element(By.TAG_NAME, "body").text
        return webpage_text
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None


def scrape_and_predict(urls):
    """Scrape multiple URLs and predict dark patterns."""
    results = {}
    driver = setup_driver()
    
    try:
        for url in urls:
            print(f"Scraping {url}...")
            webpage_text = scrape_page(url, driver)
            print(webpage_text)
            if webpage_text:
                contains_dark_patterns = predict_dark_patterns(webpage_text)
                results[url] = contains_dark_patterns
    finally:
        driver.quit()
        
    return results

def main():
    urls = [
        # "https://www.lipsum.com/",
        # "https://www.reddit.com/r/darkpatterns/comments/10tubkr/what_are_some_examples_of_dark_patterns_on/"
        "https://www.amazon.in/Daikin-Inverter-Display-Technology-MTKL50U/dp/B0BK1KS6ZD/ref=sr_1_2?_encoding=UTF8&content-id=amzn1.sym.58c90a12-100b-4a2f-8e15-7c06f1abe2be&dib=eyJ2IjoiMSJ9.LpujZ4uISPUK8sa_6yNGVTLp2_seTR9samDUOPD7O24PE2kANyrymdZzhCoNMjav4RVZpWfhS7NpfmT_1QrlbWK8IvTfXwV-qMalaE9wpOkQiW1Wl2-7r1lW2el2hgSS4-Vp0CvRYV2-HedAeu4HGDoDBdMN-FgO_NTgY_Y4z-qsOH4_cGCi4MaI5au3m1xprFGO4Y3z4TQxRrCaGjkW6QoVWai9ptEtd96K_Sac5QQ2HbvrckkgZRQUZz8F78OAd-o7_f6m5u0_hDv8-Q6lNh8wVeXzpEwE9_QCM_vSKDI.J2JkzduIQN1BWCiZlePcSwyqnUY_FnkkOfMyYedB5mk&dib_tag=se&pd_rd_r=2561953e-6359-4825-8748-1d5d2d584deb&pd_rd_w=e8wqo&pd_rd_wg=8gWP8&qid=1743102411&refinements=p_85%3A10440599031&rps=1&s=kitchen&sr=1-2&th=1"

        # Add more URLs as needed
    ]
    
    results = scrape_and_predict(urls)
    
    print("\nResults:")
    for url, contains_dark_patterns in results.items():
        print(f"URL: {url} - Contains dark patterns: {contains_dark_patterns}")

if __name__ == "__main__":
    main()

Scraping https://www.amazon.in/Daikin-Inverter-Display-Technology-MTKL50U/dp/B0BK1KS6ZD/ref=sr_1_2?_encoding=UTF8&content-id=amzn1.sym.58c90a12-100b-4a2f-8e15-7c06f1abe2be&dib=eyJ2IjoiMSJ9.LpujZ4uISPUK8sa_6yNGVTLp2_seTR9samDUOPD7O24PE2kANyrymdZzhCoNMjav4RVZpWfhS7NpfmT_1QrlbWK8IvTfXwV-qMalaE9wpOkQiW1Wl2-7r1lW2el2hgSS4-Vp0CvRYV2-HedAeu4HGDoDBdMN-FgO_NTgY_Y4z-qsOH4_cGCi4MaI5au3m1xprFGO4Y3z4TQxRrCaGjkW6QoVWai9ptEtd96K_Sac5QQ2HbvrckkgZRQUZz8F78OAd-o7_f6m5u0_hDv8-Q6lNh8wVeXzpEwE9_QCM_vSKDI.J2JkzduIQN1BWCiZlePcSwyqnUY_FnkkOfMyYedB5mk&dib_tag=se&pd_rd_r=2561953e-6359-4825-8748-1d5d2d584deb&pd_rd_w=e8wqo&pd_rd_wg=8gWP8&qid=1743102411&refinements=p_85%3A10440599031&rps=1&s=kitchen&sr=1-2&th=1...
.in
Delivering to Erode 638002
Update location
Home & Kitchen
All Categories
Alexa Skills
Amazon Devices
Amazon Fashion
Amazon Fresh
Amazon Pharmacy
Appliances
Apps & Games
Audible Audiobooks
Baby
Beauty
Books
Car & Motorbike
Clothing & Accessories
Collectibles
Computers & Accessories
Deals
Electronics
F